# Inicjalizacja kontekstu użytkownika

Cel: izolacja zasobów dla każdego uczestnika. Tworzone są schematy per użytkownik w katalogu `training_catalog`: `<user_slug>_bronze`, `<user_slug>_silver`, `<user_slug>_gold`. Uruchom tę komórkę przed dalszymi krokami w innych notebookach.

## 📁 Architektura danych w szkoleniu

Szkolenie używa **dwóch podejść do przechowywania danych** w zależności od dnia:

### Dzień 1-2: Lokalne pliki → Delta Tables
- **Źródło**: Folder `dataset/` (CSV, JSON, Parquet)
- **Zmienna**: `DATASET_BASE_PATH` 
- **Cel**: Załadowanie do Delta tables w schematach Bronze/Silver/Gold
- **Przykład**: `spark.read.csv(f"{DATASET_BASE_PATH}/customers/customers.csv")`

### Dzień 3: Unity Catalog Volumes
- **Źródło**: Unity Catalog Volumes (`/Volumes/training_catalog/default/kion_datasets`)
- **Zmienna**: `VOLUMES_BASE_PATH` (opcjonalne dla Dzień 3)
- **Cel**: Demonstracja zaawansowanych feature'ów UC (DLT, Lakeflow, Governance)
- **Przykład**: `spark.read.csv("/Volumes/training_catalog/default/kion_datasets/customers.csv")`

### Dlaczego dwa podejścia?
- **Progresja edukacyjna**: Od prostych plików (Dzień 1) → Delta Lake (Dzień 2) → Unity Catalog (Dzień 3)
- **Real-world scenarios**: W produkcji często używa się Volumes dla managed data access w UC
- **Best practices**: Dzień 3 pokazuje, jak zarządzać danymi w enterprise environment

---

In [0]:
# === Konfiguracja katalogów i schematów ===
CATALOG = "training_catalog"

# Pobierz aktualnego użytkownika (dla środowiska produkcyjnego)
# raw_user = spark.sql("SELECT current_user()").first()[0]
raw_user = "trainer"  # Dla środowiska szkoleniowego

import re
user_slug = re.sub(
    r'[^a-z0-9]', 
    '_', 
    raw_user.lower()
)

print(f"User slug: {user_slug}")

# Schematy per użytkownik (izolacja zasobów)
BRONZE_SCHEMA = f"{user_slug}_bronze"
SILVER_SCHEMA = f"{user_slug}_silver"
GOLD_SCHEMA   = f"{user_slug}_gold"

# === Ścieżka do datasetów ===
# Dla lokalnego środowiska - ścieżka do folderu dataset w tym repo
#import os
#DATASET_BASE_PATH = os.path.abspath("../dataset")

 #Dla Databricks środowiska - Unity Catalog Volume:
DATASET_BASE_PATH = "/Volumes/training_catalog/default/kion_datasets"


print(f"Dataset base path: {DATASET_BASE_PATH}")

# === Tworzenie katalogów i schematów (tylko w Databricks) ===
try:
    spark.sql(f'USE CATALOG {CATALOG}')
    
    for s in [BRONZE_SCHEMA, SILVER_SCHEMA, GOLD_SCHEMA]:
        spark.sql(f'CREATE SCHEMA IF NOT EXISTS {CATALOG}.{s}')
    
    spark.sql(f'USE SCHEMA {BRONZE_SCHEMA}')
    print("✓ Unity Catalog schemas created successfully")
except Exception as e:
    print(f"⚠ Unity Catalog not available or not configured: {e}")
    print("Continuing with default database...")

# === Wyświetl informacje o katalogu ===
print("\n=== Informacje o katalogu ===")
display(
    spark.sql(f"DESCRIBE CATALOG EXTENDED {CATALOG}")
)

# === Podsumowanie konfiguracji ===
print("\n=== Podsumowanie konfiguracji użytkownika ===")
display(
    spark.createDataFrame(
        [
            (
                raw_user, 
                f'{CATALOG}.{BRONZE_SCHEMA}', 
                f'{CATALOG}.{SILVER_SCHEMA}', 
                f'{CATALOG}.{GOLD_SCHEMA}',
                DATASET_BASE_PATH
            )
        ],
        ['user', 'bronze_schema', 'silver_schema', 'gold_schema', 'dataset_path']
    )
)

print("\n✓ Inicjalizacja zakończona pomyślnie!")
print(f"✓ Użytkownik: {raw_user}")
print(f"✓ Katalog: {CATALOG}")
print(f"✓ Schematy: {BRONZE_SCHEMA}, {SILVER_SCHEMA}, {GOLD_SCHEMA}")
print(f"✓ Ścieżka do danych: {DATASET_BASE_PATH}")
